In [ ]:
# -*- coding: utf-8 -*-

# Sources:
    # https://platform.openai.com/docs/guides/batch/rate-limits
    # https://cookbook.openai.com/examples/batch_processing
    # https://cookbook.openai.com/examples/tag_caption_images_with_gpt4v (to check)


# Importing necessary libraries

from openai import OpenAI
import pandas as pd
from IPython.display import Image, display

import base64


# API key
OPENAI_API_KEY='your-API-key-here'  # Replace with your OpenAI API key

# Initializing OpenAI client - see https://platform.openai.com/docs/quickstart?context=python
client = OpenAI(api_key=OPENAI_API_KEY)

# Load the dataset
dataset_path = "inputs.csv" 
# Read the CSV file into a DataFrame
df = pd.read_csv(dataset_path, encoding='latin-1', sep= ',')

# Create url column by concatenating the base URL with the filename
df['path'] = 'img/' + df['id']


In [ ]:
caption_system_prompt_new = '''
            You are an image classification system specialized in Cultural Ecosystem Services (CES). 
            Your task is to classify the given image into exactly one of the predefined categories below. 
            Return only the category name and nothing else. Do not provide explanations or additional text.

            Categories:
            Spiritual Services: This includes sacred or religious inspiration derived from ecosystems.
            Inspiration: This includes the use of natural motifs or artifacts in art, folklore, etc.
            Recreation and Tourism: This includes activities like hiking, fishing, and visiting natural sites.
            Heritage Values: This includes the cultural and historical significance of ecosystems and natural features.
            Cultural Identity: This includes the sense of belonging and connection to an area's natural environment.
            Aesthetic Appreciation: This includes the enjoyment of natural landscapes and the beauty of ecosystems
            
            Not Relevant: Images that are not relevant for Cultural Ecosystem Services studies. Images including photos that are too blurry, out of focus, screenshots, memes, close-up selfies that don't show any of the surrounding environment, pictures of photos, photos of phone or computer screens, 
            photos taken inside a room with or without people where nothing from the outside is visible, photos of toys, dolls, legos, etc. without any other context, photos taken inside vehicles that don't show landscapes or anything relevant from the outside, photos of music concerts, 
            wedding photos, close-ups of household elements like windows or modern doors that don't have cultural value.


            **Return only one exact category name from the list above**. Do not add any explanations, descriptions or additional information.
            '''


In [4]:
def get_caption_local(image_path, title):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()

    response = client.chat.completions.create(
        model="gpt-4.1",  # o gpt-4o
        temperature=0.2,
        max_tokens=300,
        messages=[
            {"role": "system", "content": caption_system_prompt_new},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": title},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64.b64encode(image_bytes).decode()}"
                        }
                    },
                ]
            }
        ]
    )

    return response.choices[0].message.content


In [ ]:
# load libraries
import pandas as pd
import time
import os

# Create output directory if it doesn't exist
os.makedirs('results_local', exist_ok=True)

# batch processing
batch_size = 100
num_batches = (len(df) + batch_size - 1) // batch_size

for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(df))
    batch_df = df.iloc[batch_start:batch_end]

    results = []

    for _, row in batch_df.iterrows():
        image_path = row['path']
        title = row['id']
        try:
            caption = get_caption_local(image_path, title)
        except Exception as e:
            print(f"Error en imagen {title}: {e}")
            caption = "ERROR"

        results.append({
            'id': title,
            'path': image_path,
            'caption': caption
        })

        time.sleep(2)  # Espera para no saturar la API

    # Guardar resultados del batch
    output_batch_df = pd.DataFrame(results)
    output_batch_df.to_csv(f'results_local/captions_output_batch_{i+1}.csv', index=False)

    print(f'✅ Batch {i+1}/{num_batches} guardado con {len(results)} imágenes.')